# Complete NeuroBM Tutorial: From Theory to Practice

**Educational/Research Platform for Cognitive Modeling**

This comprehensive tutorial covers the entire NeuroBM workflow from theoretical foundations to practical implementation and interpretation.

## 🎯 Tutorial Objectives

1. **Understand** Boltzmann machine theory and cognitive modeling applications
2. **Implement** complete training pipelines for different scenarios
3. **Analyze** learned representations and patterns
4. **Interpret** results with proper limitations and ethical considerations
5. **Apply** best practices for research and education

## ⚠️ Important Disclaimers

- **Educational Purpose Only**: All examples are for learning and research
- **No Clinical Applications**: Not for diagnosis, treatment, or medical decisions
- **Synthetic Data**: All data is artificially generated
- **Theoretical Models**: Simplified representations of complex phenomena
- **Validation Required**: Real applications need proper validation studies

In [ ]:
# Complete imports for the tutorial
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import sys
import yaml
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(str(Path.cwd().parent))

# NeuroBM complete imports
from neurobm.models.rbm import RestrictedBoltzmannMachine
from neurobm.models.dbm import DeepBoltzmannMachine
from neurobm.models.crbm import ConditionalRBM
from neurobm.data.synth import SyntheticDataGenerator
from neurobm.data.loaders import get_data_loader, get_sequence_loader
from neurobm.data.schema import get_schema
from neurobm.data.transforms import create_preprocessing_pipeline
from neurobm.training.loop import TrainingLoop
from neurobm.training.callbacks import get_standard_callbacks
from neurobm.training.eval import ModelEvaluator
from neurobm.interpret.saliency import SaliencyAnalyzer
from neurobm.interpret.mutual_info import MutualInformationAnalyzer
from neurobm.interpret.traversals import LatentTraverser
from neurobm.interpret.tiles import FilterVisualizer

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 NeuroBM Tutorial Environment Ready")
print(f"📱 Device: {device}")
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"\n🧠 Ready to explore cognitive modeling with Boltzmann machines!")

## Part 1: Understanding the Data

Let's start by exploring the different cognitive regimes available in NeuroBM.

In [ ]:
# Explore available regimes
regimes = ['base', 'ptsd', 'autism', 'ai_reliance']

print("🗂️ Available Cognitive Regimes:\n")

regime_info = {}
for regime in regimes:
    try:
        schema = get_schema(regime)
        features = list(schema.features.keys())
        regime_info[regime] = {
            'features': features,
            'n_features': len(features),
            'description': schema.description if hasattr(schema, 'description') else f'{regime.title()} cognitive features'
        }
        
        print(f"📋 {regime.upper()}:")
        print(f"   Features ({len(features)}): {', '.join(features)}")
        print()
    except Exception as e:
        print(f"❌ Could not load {regime}: {e}")

# Create comparison DataFrame
comparison_data = []
for regime, info in regime_info.items():
    comparison_data.append({
        'Regime': regime.title(),
        'Features': info['n_features'],
        'Focus': info['description']
    })

comparison_df = pd.DataFrame(comparison_data)
print("📊 Regime Comparison:")
print(comparison_df.to_string(index=False))

## Part 2: Model Comparison Workflow

Let's implement a complete workflow that compares different models on the same data.

In [ ]:
# Generate data for comparison
print("🔬 Generating Comparison Dataset...\n")

# Use base regime for comparison
generator = SyntheticDataGenerator('base', random_seed=42)
data = generator.generate_samples(n_samples=2000, method='skewed')
feature_names = list(generator.schema.features.keys())

# Split data
train_data = data[:1600]  # 80% for training
test_data = data[1600:]   # 20% for testing

print(f"📊 Dataset Summary:")
print(f"• Total samples: {len(data)}")
print(f"• Training samples: {len(train_data)}")
print(f"• Test samples: {len(test_data)}")
print(f"• Features: {len(feature_names)}")
print(f"• Feature names: {feature_names}")

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(train_data),
    batch_size=32,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(test_data),
    batch_size=32,
    shuffle=False
)

print(f"\n🔄 Data Loaders Created:")
print(f"• Train batches: {len(train_loader)}")
print(f"• Test batches: {len(test_loader)}")